# DRAFT: Comparing ML model (built in or same as Simba) interaction times to gold standard holdout set and stopwatch human data

NOTE: "score" will refer to binary f1, precision, and/or recall.  Relative to the target class, "Interaction".

Goal:  
- Build a classifier based on the same procedure as Simba.  
- Score via per-video cross validation during training, to estimate the generalization accuracy for unseen videos.
- Score at each step of processing:
    - Building the classifier
    - min_bought_duratin post processing
    - Kleinburg Filtering post processing
- Compare to stop watched labelled human data.
- Analyze the distribution of errors relative to each individual object, each treatment class, and each rat.
    - Ideally the errors will have no bias and be zero mean Guassian across each categorical split.

In [1]:
import numpy
import glob
import os
import functools
import math
import numpy as np
import pandas as pd

## Dataset class to encapsulate opening files and creating cv_indexes required for running cross_validation in sklearn
## on a per video basis.  We will also encapsulate handling individual Dataframes per video vs. one large Dataframe
## with all the videos.  The large Dataframe

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

from collections import namedtuple

PartialResults = namedtuple(
    'PartialResults',
    'precision recall f1 c_mat')
FullResults = namedtuple(
    'FullResults',
    'model test_results train_results total_time')

def build_partial_result(y_true, y_pred):
    try:
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    except:
        # must be multi class
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    # precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    c_mat = confusion_matrix(y_true, y_pred)
    return PartialResults(precision, recall, f1, c_mat)

class ModelResultLabels(object):
    """ Another data class for holding results """
    def __init__(self, func, y_binary, y_multi, y_prob, binary_results, multi_label_results):
        self._func = func # just in case, will use name for printing
        self.y_binary = y_binary
        self.y_multi = y_multi
        self.y_prob = y_prob # Probability will always be relative to the binary labels.
        self.binary_results = binary_results
        self.multi_label_results = multi_label_results

class PerVideoDataClass(object):
    """ The Dataset class will aggregate 1 instance of this class for each video it manages. """
    def __init__(self, file_path, df, x_extractor, y_binary_label_extractor, y_multi_label_extractor):
        self._full_path = file_path # just in case
        self.video_name = os.path.basename(file_path) # the video name contains meta data about animal and treatment group etc.
        self.X = x_extractor(df)
        self.y_binary = y_binary_label_extractor(df) # The test labels.
        self.y_multi = y_multi_label_extractor(df, self.y_binary)
        self._y_multi_label_extractor = functools.partial(y_multi_label_extractor, df) # need for later, the rest can go.
        import collections
        self.model_result_labels = collections.OrderedDict()
    
    def apply_and_record_postprocessing_step(self, func, y_binary, y_prob=None):
        """ Func was used to produce y_binary, we record the results for analysis later. """
        if callable(func):
            key = func.__name__
        else:
            key = func.__class__.__name__
        y_multi = self._y_multi_label_extractor(y_binary)
        self.model_result_labels[key] = ModelResultLabels(
            func, y_binary, y_multi, y_prob, 
            build_partial_result(self.y_binary, y_binary),
            build_partial_result(self.y_multi, y_multi))
    
    def __str__(self):
        return f'''
        Video name: {self.video_name}
        Results stored: {self.model_result_labels.keys()}
        '''

    def __repr__(self):
        return str(self)

class Dataset(object):
    def __init__(self, input_files, x_extractor, y_binary_label_extractor, y_multi_label_extractor):
        # assert isinstance(input_files, list)
        self.data_files = input_files
        dfs = [pd.read_csv(f, index_col=0) for f in input_files]
        self._original_dfs = dfs # Just in case
        self.per_video_data_classes = [
            PerVideoDataClass(
                file_path,
                df, 
                x_extractor=x_extractor,
                y_binary_label_extractor=y_binary_label_extractor,
                y_multi_label_extractor=y_multi_label_extractor)
            for file_path, df in zip(input_files, dfs)
        ]

    def __str__(self):
        files_str = '\n\t'.join(self.data_files)
        paths_str = f'Data files: {files_str}'
        return f'{self.__class__.__name__}:\n' \
               f'Number of files: {len(self.data_files)}\n' \
               f'Paths: {paths_str}'

## Pre Processing:
## All we need to do is select the desired input features, and the target columns

## Define specific extraction methods.
def build_Y_get_interaction(col='Interaction'):
    """ For the objects datasets """
    def Y_get_interaction(df: pd.DataFrame):
        ys = df[col]
        ys = ys.fillna(value=0.0)
        return ys.values
    return Y_get_interaction


def build_Y_get_multi_label(ROI_features):
    def Y_get_multi_label(df, ys_binary):
        """ ROI features are binary and encode which (if any)"""
        ROI_values = df[ROI_features].values
        # Need to create a default category. Right now ROI_values is formatted like:
        # [[0,0,0,0,0,0], # No interaction
        #  [0,0,1,0,0,0], # Interaction with third object.
        #  ...]
        # We want argmax to return 0 if all the ROI_values are 0.
        # And we want argmax to return the number of the column that is non-zero otherwise.
        # So we add a column of 0.5 (can be any value between 1 and 0) to be the first column.
        # Now we will get 0 by default, and the correct (in the 1 based indexing sence) column number
        # for the object which was interacted with.
        assert numpy.all((ys_binary == 1) | (ys_binary == 0))
        assert numpy.all((ROI_values == 1) | (ROI_values == 0))

        # TODO: AARONT: This isn't right either.  We need to use min distance and that label instead!

        default_column = numpy.repeat(0.5, ROI_values.shape[0])
        ROI_values = numpy.hstack([default_column.reshape(-1,1), ROI_values])
        # Now ROI_values looks like this:
        # [[0.5,0,0,0,0,0,0], # No interaction
        #  [0.5,0,0,1,0,0,0], # Interaction with third object.
        #  ...]

        multi_label = numpy.argmax(ROI_values, axis=1, )
        return multi_label
    return Y_get_multi_label

# x extractors
def build_X_extractor(input_features):
    def X_extractor(df):
        # input_features should be a set of columns known to be in the Dataframe.
        return df[input_features]
    return X_extractor






In [2]:

interaction_features = [
    'Interaction',
    'Probability_Interaction'
]

raw_dlc_features_only = [
    "Ear_left_p",
    "Ear_left_x",
    "Ear_left_y",

    "Ear_right_p",
    "Ear_right_x",
    "Ear_right_y",

    "Lat_left_p",
    "Lat_left_x",
    "Lat_left_y",

    "Lat_right_p",
    "Lat_right_x",
    "Lat_right_y",

    "Center_p",
    "Center_x",
    "Center_y",

    "Nose_p",
    "Nose_x",
    "Nose_y",

    "Tail_base_p",
    "Tail_base_x",
    "Tail_base_y",

    "Tail_end_x",
    "Tail_end_y",
    "Tail_end_p",
]

## Load the data.  Pre engineered features created in Simba with Region of Interest (ROI) data included.
## ROI data is centered on each of 6 objects of interest.
training_simba_csv_files = glob.glob(
    os.path.join('hackathon', 'Iteration_2_withROI', 'targets_inserted', '*.csv'))
holdout_simba_csv_files = glob.glob(
    os.path.join('hackathon', 'Iteration_2_withROI', 'hold_dataset', '*.csv'))

# # TEMP: Only 1 each for now while coding
# training_simba_csv_files = training_simba_csv_files[0:2]
# holdout_simba_csv_files = holdout_simba_csv_files[0:2]

# temporary Dataframe so we can read the columns.
temp_df = pd.read_csv(training_simba_csv_files[0], index_col=0)

exclude_columns = interaction_features + raw_dlc_features_only
input_features = [col for col in temp_df.columns if col not in exclude_columns]
x_extractor = build_X_extractor(input_features)
y_binary_label_extractor = build_Y_get_interaction(col='Interaction')

import re
ROI_re = re.compile(r'^Stimulus [0-9] Animal_[0-9]+ in zone$')
ROI_features = [col for col in temp_df.columns if ROI_re.match(col)]
print('ROI_features:', ROI_features)
y_multi_label_extractor = build_Y_get_multi_label(ROI_features)

# def apply_and_record_postprocessing_step(self, func, y_binary): # Don't forget to use this later

training_dataset = Dataset(training_simba_csv_files, x_extractor, y_binary_label_extractor, y_multi_label_extractor)
# We don't make cv indexes for the holdout set because we will only be using this to verify
# model performance on unseen videos.
holdout_dataset = Dataset(holdout_simba_csv_files, x_extractor, y_binary_label_extractor, y_multi_label_extractor)
print(f'Training dataset: {training_dataset}')
print(f'Holdout dataset: {holdout_dataset}')


ROI_features: ['Stimulus 1 Animal_1 in zone', 'Stimulus 2 Animal_1 in zone', 'Stimulus 3 Animal_1 in zone', 'Stimulus 4 Animal_1 in zone', 'Stimulus 5 Animal_1 in zone', 'Stimulus 6 Animal_1 in zone']
Training dataset: Dataset:
Number of files: 27
Paths: Data files: hackathon/Iteration_2_withROI/targets_inserted/2022-06-12_NOD_IOT_18.csv
	hackathon/Iteration_2_withROI/targets_inserted/2022-06-24_NOB_IOT_22.csv
	hackathon/Iteration_2_withROI/targets_inserted/08092021_IOT_Rat11_12.csv
	hackathon/Iteration_2_withROI/targets_inserted/08092021_DOT_Rat3_4.csv
	hackathon/Iteration_2_withROI/targets_inserted/08092021_DOT_Rat9_10.csv
	hackathon/Iteration_2_withROI/targets_inserted/08102021_IOT_Rat3_4.csv
	hackathon/Iteration_2_withROI/targets_inserted/2022-06-21_NOB_IOT_23.csv
	hackathon/Iteration_2_withROI/targets_inserted/08_11_2021_DOT_Rat7_8.csv
	hackathon/Iteration_2_withROI/targets_inserted/2022-06-12_NOD_IOT_14.csv
	hackathon/Iteration_2_withROI/targets_inserted/08102021_DOT_Rat11_12.csv

## Fit the model
Okay, data is loaded, features and labels extracted and stored in two Dataset objects.
No we are ready to fit a model, and get some performance metrics!

In [3]:
from xgboost import XGBClassifier

# From a previous GridSearchCV:
# 
# Finished training model; best_score: 0.4699529225702816; 
# best_estimator: XGBClassifier(base_score=0.5, booster='gbtree', gamma=0.0,
#               grow_policy='lossguide', interaction_constraints=None,
#               learning_rate=0.3, max_delta_step=1, max_depth=4,
#               min_child_weight=0, missing=None, n_estimators=300, n_jobs=14,
#               nthread=None, num_parallel_tree=1, objective='binary:logistic',
#               random_state=0, reg_alpha=0, reg_lambda=1,
#               sampling_method='gradient_based',
#               scale_pos_weight=4.296046179280971, seed=None, silent=None,
#               subsample=0.5, tree_method='gpu_hist', verbosity=1)
# Best grid search params: {'gamma': 0.0, 'learning_rate': 0.3, 'max_delta_step': 1, 'max_depth': 4, 'min_child_weight': 0, 'n_estimators': 300, 'scale_pos_weight': 4.296046179280971}
#
# NOTE: scale_pos_weight will be fit with a bit of a trick for xgboost, based on the frequency of labels
#       in the input dataset.


# 1. Get all training Xs and ys from the dataset.  Use proportion in ys to set scale_pos_weight

Xs = []
ys = []
for per_video_data in training_dataset.per_video_data_classes:
    Xs.append(per_video_data.X)
    ys.append(per_video_data.y_binary) # use binary ys for training and predictions.

Xs = pd.concat(Xs)
# Xs = numpy.vstack(Xs)
ys = numpy.hstack(ys)

# If a path is provided we load a model file from disk, otherwise we fit a new one.
model_path = '' # Put path to your model here

if model_path:
    import pickle
    with open(model_path, 'wb') as f:
        clf = pickle.load(f, protocal=pickle.HIGHEST_PROTOCOL)
else:
  # Fit new model
    new_scale_pos_weights = (ys == 0).sum() / (ys >= 1).sum()
    print(new_scale_pos_weights)
    clf = XGBClassifier(base_score=0.5, booster='gbtree', gamma=0.0,
                        grow_policy='lossguide', interaction_constraints=None,
                        learning_rate=0.3, max_delta_step=1, max_depth=4,
                        min_child_weight=0, n_estimators=300, n_jobs=10,
                        nthread=None, num_parallel_tree=1, 
                        objective='binary:logistic',
                        # objective='binary:logitraw',
                        random_state=0, reg_alpha=0, reg_lambda=1,
                        sampling_method='gradient_based',
                        scale_pos_weight=new_scale_pos_weights, seed=42, silent=None,
                        subsample=0.5, tree_method='gpu_hist', verbosity=1)

#   from sklearn.ensemble import RandomForestClassifier
#   clf = RandomForestClassifier(
#                 n_estimators=200,
#                 bootstrap=True,
#                 verbose=0, # 1 if you want to see jobs etc
#                 n_jobs=-1,
#                 criterion='gini',  # Gini is standard, shouldn't be a huge factor
#                 min_samples_leaf=2,
#                 max_features='sqrt',
#                 # max_depth=15,  # LIMIT MAX DEPTH!!  Runtime AND generalization error should improve drastically
#                 random_state=42,
#                 #     ccp_alpha=0.005, # NEW PARAMETER, I NEED TO DEFINE MY EXPERIMENT SETUPS BETTER, AND STORE SOME RESULTS!!
#                 # Probably need to whip up a database again, that's the only way I have been able to navigate this in the past
#                 # Alternatively I could very carefully define my experiments, and then run them all in a batch and create a
#                 # meaningful report.  This is probably the best way to proceed.  It will lead to the most robust iteration
#                 # and progress.
#                 # class_weight='balanced',  # balance weights at nodes based on class frequencies
#               )

clf.fit(Xs, ys)

4.1051296735662275


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.0, gpu_id=0,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints='', learning_rate=0.3, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=1,
              max_depth=4, max_leaves=0, min_child_weight=0, missing=nan,
              monotone_constraints='()', n_estimators=300, n_jobs=10,
              nthread=10, num_parallel_tree=1, predictor='auto', ...)

# Record Results and Apply Post Processing
Now we have a trained model.  We're going to define some post-processing functions that match what is provided in Simba and then record the results at each step (classifier labels, labels after min_bought_duration smoothing, 
and labels after Kleinburg Filtering).  Then we will calculate classifier performance at each of these steps.
Finally we will calculate total interaction times for each video, then aggregate the results per animal, per treatment group, and per object.

In [4]:
def kleinberg(offsets: np.ndarray, s=2, gamma=1):
    """ TODO: Cite/give credit to Simba devs for implementation"""
    if s <= 1:
        raise ValueError("s must be greater than 1'!")
    if gamma <= 0:
        raise ValueError("gamma must be positive!")
    if len(offsets) < 1:
        raise ValueError("offsets must be non-empty!")

    assert offsets.ndim == 1
    offsets = np.array(offsets, dtype=object)

    if offsets.size == 1:
        bursts = np.array([0, offsets[0], offsets[0]], ndmin=2, dtype=object)
        return bursts

    # offsets = np.sort(offsets)
    gaps = np.diff(offsets)

    if not np.all(gaps):
        raise ValueError("Input cannot contain events with zero time between!")

    T = np.sum(gaps)
    n = np.size(gaps)

    g_hat = T / n

    k = int(math.ceil(float(1 + math.log(T, s) + math.log(1 / np.amin(gaps), s))))

    gamma_log_n = gamma * math.log(n)

    def tau(i, j):
        if i >= j:
            return 0
        else:
            return (j - i) * gamma_log_n

    alpha_function = np.vectorize(lambda x: s ** x / g_hat)
    alpha = alpha_function(np.arange(k))

    def f(j, x): # The exponential dist function for index j
        return alpha[j] * math.exp(-alpha[j] * x)

    C = np.repeat(float("inf"), k)
    C[0] = 0

    q = np.empty((k, 0))
    for t in range(n):
        C_prime = np.repeat(float("inf"), k)
        q_prime = np.empty((k, t + 1))
        q_prime.fill(np.nan)

        for j in range(k):
            cost_function = np.vectorize(lambda x: C[x] + tau(x, j))
            cost = cost_function(np.arange(0, k))

            el = np.argmin(cost)

            if f(j, gaps[t]) > 0:
                C_prime[j] = cost[el] - math.log(f(j, gaps[t]))

            if t > 0:
                q_prime[j, :t] = q[el, :]

            q_prime[j, t] = j + 1

        C = C_prime
        q = q_prime

    j = np.argmin(C)
    q = q[j, :]

    prev_q = 0

    N = 0
    for t in range(n):
        if q[t] > prev_q:
            N = N + q[t] - prev_q
        prev_q = q[t]

    # bursts = np.vstack([np.repeat(np.nan, N), np.repeat(offsets[0], N), np.repeat(offsets[0], N)]).transpose()
    bursts = np.array([np.repeat(np.nan, N), np.repeat(offsets[0], N), np.repeat(offsets[0], N)], ndmin=2, dtype=object).transpose()
    burst_counter = -1
    prev_q = 0
    stack = np.repeat(np.nan, N)
    stack_counter = -1
    for t in range(n):
        if q[t] > prev_q:
            num_levels_opened = q[t] - prev_q
            for i in range(int(num_levels_opened)):
                burst_counter += 1
                bursts[burst_counter, 0] = prev_q + i
                bursts[burst_counter, 1] = offsets[t]
                stack_counter += 1
                stack[stack_counter] = burst_counter
        elif q[t] < prev_q:
            num_levels_closed = prev_q - q[t]
            for i in range(int(num_levels_closed)):
                bursts[int(stack[stack_counter]), 2] = offsets[t]
                stack_counter -= 1
        prev_q = q[t]

    while stack_counter >= 0:
        bursts[int(stack[stack_counter]), 2] = offsets[n]
        stack_counter -= 1

    return bursts

def build_Y_post_processor_klienberg_filtering():
    def Y_post_processor_klienberg_filtering(y_pred): #, _df: pd.DataFrame):
        # df = _df.copy(deep=True)
        # AARONT: TODO: Had 'math domain error downstream here, would have to fix that!  Turning off'
        # from simba.Kleinberg_burst_analysis import kleinberg
        # kleinberg filtering setup args etc
        classifierName = 'Interaction'
        logs_path = 'TEMP_kburg_logs_path'
        os.makedirs(logs_path, exist_ok=True)
        hierarchy = 1
        ## Trying to do this without requiring the dataframe...
        # assert len(df) == len(y_pred)
        # currDf = df[y_pred == 1]
        # offsets = list(currDf.index.values)
        # split into offsets by video
        ## AARONT: This will cause issues if we used for example undersampling upstream, since we
        #          aren't using the video indexes anymore.  But we shouldn't be doing that upstream,
        #          so it shouldn't be a problem.  And even if someone did that would produce very strange
        #          results and shouldn't be done.
        offsets = numpy.where(y_pred == 1)[0]

        # kleinberg apply algorithm
        # print(f'offsets: {offsets}')
        # print(f'df cols: {df.columns}')
        # AARONT: TODO: I think the math domain error is due to the offsets calculation, they need to have some spacing
        #               or something like that and are not getting the spacing they need!
        # From the paper: Adjusting 'b' controls inertia that keeps automaton in it's current state (which arg is b?)
        #
        kleinbergBouts = kleinberg(offsets, s=2.0, gamma=0.3) # TODO: Params?
        print(f'AARONT: k-filtering bouts (head): {kleinbergBouts[0:3]}')
        kleinbergDf = pd.DataFrame(kleinbergBouts, columns=['Hierarchy', 'Start', 'Stop'])
        kleinbergDf['Stop'] += 1
        file_name = 'Kleinberg_log_' + classifierName + '.csv'
        logs_file_name = os.path.join(logs_path, file_name)
        kleinbergDf.to_csv(logs_file_name)
        kleinbergDf_2 = kleinbergDf[kleinbergDf['Hierarchy'] == hierarchy].reset_index(drop=True)
        # df[classifierName] = 0
        new_y_pred = numpy.zeros_like(y_pred)
        for index, row in kleinbergDf_2.iterrows():
            rangeList = list(range(int(row['Start']), int(row['Stop'])))
            new_y_pred[rangeList] = 1
            # for frame in rangeList:
                # df.at[frame, classifierName] = 1
        # y_pred = df[classifierName].values
        return new_y_pred
    return Y_post_processor_klienberg_filtering


def FROM_SIMBA_plug_holes_shortest_bout(y_pred, min_bout_duration): #, fps=None, shortest_bout=None):
    """
    First, find all patterns like `1 0 0 0 ... 0 0 0 1` where the number of frames that are zeros is
    less than or equal to min_bout_duration and fill them with 1's.
    Then find all patterns like `0 1 1 1 ... 1 1 1 0` with the same length specification, and fill those
    with 0's.
    """
    col_name = 'y_pred_col'
    data_df = pd.DataFrame(y_pred, columns=[col_name])
    # frames_to_plug = int(int(fps) * int(shortest_bout) / 1000)
    frames_to_plug_lst = list(range(1, min_bout_duration + 1))
    frames_to_plug_lst.reverse()
    patternListofLists, negPatternListofList = [], []
    for k in frames_to_plug_lst:
        zerosInList, oneInlist = [0] * k, [1] * k
        currList = [1]
        currList.extend(zerosInList)
        currList.extend([1])
        currListNeg = [0]
        currListNeg.extend(oneInlist)
        currListNeg.extend([0])
        patternListofLists.append(currList)
        negPatternListofList.append(currListNeg)
    fill_patterns = numpy.asarray(patternListofLists, dtype=object)
    remove_patterns = numpy.asarray(negPatternListofList, dtype=object)

    for currPattern in fill_patterns:
        n_obs = len(currPattern)
        data_df['rolling_match'] = (data_df[col_name].rolling(window=n_obs, min_periods=n_obs)
                                    .apply(lambda x: (x == currPattern).all(), raw=True)
                                    .mask(lambda x: x == 0)
                                    .bfill(limit=n_obs - 1)
                                    .fillna(0)
                                    .astype(bool)
                                    )
        data_df.loc[data_df['rolling_match'] == True, col_name] = 1
        data_df = data_df.drop(['rolling_match'], axis=1)

    for currPattern in remove_patterns:
        n_obs = len(currPattern)
        data_df['rolling_match'] = (data_df[col_name].rolling(window=n_obs, min_periods=n_obs)
                                    .apply(lambda x: (x == currPattern).all(), raw=True)
                                    .mask(lambda x: x == 0)
                                    .bfill(limit=n_obs - 1)
                                    .fillna(0)
                                    .astype(bool)
                                    )
        data_df.loc[data_df['rolling_match'] == True, col_name] = 0
        data_df = data_df.drop(['rolling_match'], axis=1)

    return data_df[col_name]


def build_Y_post_processor_min_bought_duration(min_bout_duration):
    def Y_post_processor_min_bought_duration(y_pred: numpy.ndarray):
        """ given y_pred a vector of binary predictions, enforce a minimum number of
        concurrent predictions """
        assert numpy.all((y_pred == 1) | (y_pred == 0)), f'ERROR: y_pred must be a binary vector.  Got this instead: {y_pred}'
        assert isinstance(min_bout_duration, int)

        # print(f'y_pred BEFORE min_bought: (sum is {numpy.sum(y_pred)}; {y_pred}')
        y_pred = FROM_SIMBA_plug_holes_shortest_bout(y_pred, min_bout_duration)
        # print(f'y_pred AFTER min_bought: (sum is {numpy.sum(y_pred)}; {y_pred}')
        return y_pred
    return Y_post_processor_min_bought_duration


Y_post_processor_min_bought_duration = build_Y_post_processor_min_bought_duration(10) # TODO: What is the frame rate again??
Y_post_processor_klienberg_filtering = build_Y_post_processor_klienberg_filtering()

# Store the results of applying the base classifier to the data

def record_results(dataset):
    for per_video_data in dataset.per_video_data_classes:
        X = per_video_data.X
        y_pred = clf.predict(X)
        y_prob = clf.predict_proba(X)
        per_video_data.apply_and_record_postprocessing_step(clf, y_pred, y_prob)
        y_pred = Y_post_processor_min_bought_duration(y_pred)
        per_video_data.apply_and_record_postprocessing_step(Y_post_processor_min_bought_duration, y_pred)
        y_pred = Y_post_processor_klienberg_filtering(y_pred)
        per_video_data.apply_and_record_postprocessing_step(Y_post_processor_klienberg_filtering, y_pred)
        print('Recorded data for:', per_video_data)
    

record_results(training_dataset)
record_results(holdout_dataset)

AARONT: k-filtering bouts (head): [[0 77 5211]
 [1 77 139]
 [2 77 139]]
Recorded data for: 
        Video name: 2022-06-12_NOD_IOT_18.csv
        Results stored: odict_keys(['XGBClassifier', 'Y_post_processor_min_bought_duration', 'Y_post_processor_klienberg_filtering'])
        
AARONT: k-filtering bouts (head): [[0 14 8928]
 [1 14 38]
 [2 14 38]]
Recorded data for: 
        Video name: 2022-06-24_NOB_IOT_22.csv
        Results stored: odict_keys(['XGBClassifier', 'Y_post_processor_min_bought_duration', 'Y_post_processor_klienberg_filtering'])
        
AARONT: k-filtering bouts (head): [[0 383 7045]
 [1 383 413]
 [2 383 413]]
Recorded data for: 
        Video name: 08092021_IOT_Rat11_12.csv
        Results stored: odict_keys(['XGBClassifier', 'Y_post_processor_min_bought_duration', 'Y_post_processor_klienberg_filtering'])
        
AARONT: k-filtering bouts (head): [[0 499 7199]
 [1 499 518]
 [2 499 518]]
Recorded data for: 
        Video name: 08092021_DOT_Rat3_4.csv
        Results s

In [6]:

## These are the interfaces.
# PartialResults = namedtuple(
#     'PartialResults',
#     'precision recall f1 c_mat X y_true y_pred')
# FullResults = namedtuple(
#     'FullResults',
#     'model test_results train_results total_time')
#
# def build_partial_result(y_true, y_pred):
#     try:
#         precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
#     except:
#         # must be multi class
#         precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
#     # precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
#     c_mat = confusion_matrix(y_true, y_pred)
#     return PartialResults(precision, recall, f1, c_mat, X, y_true, y_pred)

# class ModelResultLabels(object):
#     """ Another data class for holding results """
#     def __init__(self, func, y_binary, y_multi, y_prob, binary_results, multi_label_results):
#         self._func = func # just in case, will use name for printing
#         self.y_binary = y_binary
#         self.y_multi = y_multi
#         self.y_prob = y_prob # Probability will always be relative to the binary labels.
#         self.binary_results = binary_results
#         self.multi_label_results = multi_label_results
def print_dataset_scores(dataset: Dataset):
    for per_video_data in dataset.per_video_data_classes:
        print(per_video_data.video_name)
        for func_name, results in per_video_data.model_result_labels.items():
            print(func_name)
            print(results.binary_results)
            print(results.multi_label_results)

print()
print('*' * 50)
print('TRAINING SCORES')
print_dataset_scores(training_dataset)
print()
print('*' * 50)
print('HOLDOUT SCORES')
print_dataset_scores(holdout_dataset)



**************************************************
TRAINING SCORES
2022-06-12_NOD_IOT_18.csv
XGBClassifier
PartialResults(precision=0.8203976908274535, recall=0.9937839937839937, f1=0.898805340829234, c_mat=array([[3833,  280],
       [   8, 1279]]))
PartialResults(precision=1.0, recall=1.0, f1=1.0, c_mat=array([[3453,    0,    0,    0,    0,    0,    0],
       [   0,  292,    0,    0,    0,    0,    0],
       [   0,    0,  426,    0,    0,    0,    0],
       [   0,    0,    0,  170,    0,    0,    0],
       [   0,    0,    0,    0,  517,    0,    0],
       [   0,    0,    0,    0,    0,  240,    0],
       [   0,    0,    0,    0,    0,    0,  302]]))
Y_post_processor_min_bought_duration
PartialResults(precision=0.7577981651376147, recall=0.9627039627039627, f1=0.8480492813141683, c_mat=array([[3717,  396],
       [  48, 1239]]))
PartialResults(precision=1.0, recall=1.0, f1=1.0, c_mat=array([[3453,    0,    0,    0,    0,    0,    0],
       [   0,  292,    0,    0,    0,    0, 